In [15]:
import os
from sys import platform
from Bio import SeqIO

species  = ['human', 'mouse']
salias = {'human' : 'Homo_sapiens', 'mouse' : 'Mus_musculus'}
genes = ['IGHV', 'IGHD', 'IGHJ',
         'IGKV', 'IGKJ',
         'IGLV', 'IGLJ',
         'TRBV', 'TRBD', 'TRBJ',
         'TRAV', 'TRAJ',
         'TRDV', 'TRDD', 'TRDJ',
         'TRGV', 'TRGJ']

if platform == 'linux' or platform == 'linux2':
    MAKEBLASTDB_CMD = './makeblastdb_linux'
elif platform == 'darwin':
    MAKEBLASTDB_CMD = './makeblastdb_darwin'
elif platform == 'win32':
    raise 'Cannot run on Windows'

In [17]:
def get_url(s, g):
    return f'https://www.imgt.org/download/V-QUEST/IMGT_V-QUEST_reference_directory/{salias[s]}/{g[:2]}/{g}.fasta'

def get_name(s, g):
    return f'{s}.{g[3]}'

db_files = {get_name(s, g) for s in species for g in genes}

for s in species:
    for g in genes:
        os.system(f'wget {get_url(s, g)} -O ->> {get_name(s, g)}.imgt')

for f in db_files:
    os.system(f'./edit_imgt_file.pl {f}.imgt > {f}')
    seen = set()
    records = []
    for record in SeqIO.parse(f'{f}', 'fasta'):  
        if record.seq not in seen:
            seen.add(record.seq)
            records.append(record)
    SeqIO.write(records, f'{f}', "fasta")
    os.system(f'rm {f}.imgt')    
    os.system(f'{MAKEBLASTDB_CMD} -parse_seqids -dbtype nucl -in {f}')
    os.system(f'rm {f}')

--2023-10-20 23:41:44--  https://www.imgt.org/download/V-QUEST/IMGT_V-QUEST_reference_directory/Homo_sapiens/IG/IGHV.fasta
Resolving www.imgt.org (www.imgt.org)... 195.83.84.12
Connecting to www.imgt.org (www.imgt.org)|195.83.84.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 183959 (180K) [text/plain]
Saving to: ‘STDOUT’

     0K .......... .......... .......... .......... .......... 27%  519K 0s
    50K .......... .......... .......... .......... .......... 55% 1.25M 0s
   100K .......... .......... .......... .......... .......... 83% 3.90M 0s
   150K .......... .......... .........                       100% 12.7M=0.2s

2023-10-20 23:41:44 (1.17 MB/s) - written to stdout [183959/183959]

--2023-10-20 23:41:44--  https://www.imgt.org/download/V-QUEST/IMGT_V-QUEST_reference_directory/Homo_sapiens/IG/IGHD.fasta
Resolving www.imgt.org (www.imgt.org)... 195.83.84.12
Connecting to www.imgt.org (www.imgt.org)|195.83.84.12|:443... connected.
HTTP request sent, 



Building a new DB, current time: 10/20/2023 23:41:55
New DB name:   /Users/mikesh/vcs/pyigmap/igmap/external/igblast_database/database/mouse.D
New DB title:  mouse.D
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 34 sequences in 0.00331116 seconds.




Building a new DB, current time: 10/20/2023 23:41:56
New DB name:   /Users/mikesh/vcs/pyigmap/igmap/external/igblast_database/database/human.D
New DB title:  human.D
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 46 sequences in 0.00331783 seconds.




Building a new DB, current time: 10/20/2023 23:41:57
New DB name:   /Users/mikesh/vcs/pyigmap/igmap/external/igblast_database/database/mouse.V
New DB title:  mouse.V
Sequence type: Nucleotide
Keep MBits: T
Maximum file size: 3000000000B
Adding sequences from FASTA; added 936 sequences in 0.017359 seconds.




Building a new DB, current time: 10/20/2023 23:41:58
New DB nam

In [23]:
os.environ['IGDATA'] = os.path.abspath('..')
species = 'human'
cmd = ' '.join([f'-c_region_db $IGDATA/database/ncbi_{species}_c_genes',
f'-germline_db_V $IGDATA/database/{species}.V',
f'-germline_db_D $IGDATA/database/{species}.D',
f'-germline_db_J $IGDATA/database/{species}.J',
f'-organism {species}',
f'-auxiliary_data $IGDATA/optional_file/{species}_gl.aux',
f'-show_translation -outfmt 19'])
seq = "CAGGCTGAGGACGAGGCTGATTATTACTGCAGTTCATATAGAGGCAGCGCCACTTTCGAGGTGGTGTTCGGCGGAG"
!cd ../.. && echo "{seq}" | bin/igblastn_darwin {cmd}

BLAST Database error: No alias or index file found for nucleotide database [/Users/mikesh/vcs/pyigmap/igmap/external/igblast_database/database/ncbi_1human_c_genes] in search path [/Users/mikesh/vcs/pyigmap/igmap/external::]
